# Text analytics: n-Grams Language Models

### i) Implement and train a bigram and a trigram language model

* The `corpus` used to both train and test the bigram and a trigram language models is from http://www.statmt.org/europarl/ (file: source release (text files), [only the English corpus is used 377.7 MB]). It has 65,706,989 tokens in 2,279,736 sentences and 150,120 unique tokens. 

* The first `70%` of the total sentences was used as `training set`, the next `10%` was used as `development set` (fine-tuning) and the last `20%` of the sentences was used as `test set` to examine how well each model performs.
 
* A `list` of all the tokens in the training set was built followed by a `counter` of these tokens.
* Using the above-mentioned `counter` the  tokens that occur less than 10 times in the training set was removed from the counter.
* The remaining key items of the counter was used as `vocabulary` for both models with size |V|=25,589 words.
* Every token in each data set (train, development, test) which is not present in the vocabulary was replaced by the special token `*UNK*`.
* All unigrams, bigrams and trigrams in the training set was counted. In order to build the bigrams counter it was assumed that each sentence starts with the pseudo-token `*start*` (or the pseudo-tokens `*start1*`, `*start2*` for the trigram model) and ends with `*end*`.


In [1]:
# imports 
from nltk.tokenize import TweetTokenizer
from nltk import sent_tokenize
from collections import Counter
from nltk.util import ngrams
import numpy as np
tweet_wt = TweetTokenizer()

from random import sample,seed
import math
import os,glob

In [2]:
text=''
for filename in glob.glob(os.path.join('txt/en', '*.txt')):
      with open(filename, 'r') as f:
            text += f.read()

In [3]:
# make text a list of  sentences
sentences = sent_tokenize(text)

# test, development and test sets
train_sentences = sentences[:int(len(sentences)*0.7)]
dev_sentences = sentences[int(len(sentences)*0.7):int(len(sentences)*0.8)]
test_sentences = sentences[int(len(sentences)*0.8):]

# test, development and test sets tokenazation
train_sentences_tokenized = []
for sent in train_sentences: 
    sent_tok = tweet_wt.tokenize(sent)
    train_sentences_tokenized.append(sent_tok)
    
dev_sentences_tokenized = []
for sent in dev_sentences:
    sent_tok = tweet_wt.tokenize(sent)
    dev_sentences_tokenized.append(sent_tok)

test_sentences_tokenized = []
for sent in test_sentences:
    sent_tok = tweet_wt.tokenize(sent)
    test_sentences_tokenized.append(sent_tok)

In [4]:
#tokens = tweet_wt.tokenize(text)

# total tokens in train set
train_tokens = []
for sent in train_sentences_tokenized:
    for tok in sent:
        train_tokens.append(tok)

In [5]:
# dictionary {token:number of occurances in train set}
count = Counter(train_tokens)

# remove tokens with less than 10 occurances
count2 = {key:val for key, val in count.items() if val >= 10}

vocabulary = list(set(count2.keys()))

In [6]:
# Replace all out-of-vocabular (OOV) tokens 
for sent in train_sentences_tokenized:
    for i in range(len(sent)):
        if sent[i] not in count2.keys():
            sent[i] = '*UNK*'
        
for sent in dev_sentences_tokenized:
    for i in range(len(sent)):
        if sent[i] not in count2.keys():
            sent[i] = '*UNK*'
            
for sent in test_sentences_tokenized:
    for i in range(len(sent)):
        if sent[i] not in count2.keys():
            sent[i] = '*UNK*'

In [7]:
unigram_counter = Counter()
bigram_counter  = Counter()
trigram_counter = Counter()

# Train the models 
for sent in train_sentences_tokenized:
    unigram_counter.update([gram for gram in ngrams(sent, 1, pad_left=True, pad_right=True,
                                                   left_pad_symbol='*start*',right_pad_symbol='*end*') ])
    
    bigram_counter.update([gram for gram in ngrams(sent, 2, pad_left=True, pad_right=True,
                                                   left_pad_symbol='*start*',right_pad_symbol='*end*') ])
    
    trigram_counter.update([gram for gram in ngrams(sent, 3, pad_left=True, pad_right=True,
                                                   left_pad_symbol='*start*',right_pad_symbol='*end*') ])


### ii) Probabilities  of correct VS incorrect unknown sentences based on the trained models
Check the log-probabilities that the trained models return when given (correct) sentences from the test subset vs. (incorrect) sentences of the same length (in words) consisting of randomly selected vocabulary words.

* In order to check how well the trained language models perform, we will compare the probabilities of correct sentences versus incorrect ones (same but shuffled words) using Laplace smoothing.

The log-probability of a given sentence with *k* words using bi-gram model with Laplace smoothing is given by this type:


### $log_2P_{Bigram} = log_2\frac{C(w_1|*start*) + a} { C(*start*)+ a|V|}+ log_2\frac{C(w_2|w_1) + a} {C(w_1)+ a|V|}+ \dots + log_2\frac{C(*end*|w_k) + a} {C(w_k)+ a|V|}  $



and the corresponding type for a tri-gram model is:


### $log_2P_{Trigram} = log_2\frac{C(w_1|*start1*,*start2*) + a} {C(*start1*,*start2*)+ a|V|}+ log_2\frac{C(w_2|*start2*,w_1) + a} {C(*start2*,w_1)+ a|V|}+ \dots + log_2\frac{C(*end2*|w_k,*end1*) + a} {C(w_k,*end1*)+ a|V|}  $


* $ C(w_1,w_2) $ : bigram count
* $ C(w_1) $ : unigram count
* $ 0 \leq\alpha \leq1 $ :  smoothing hyper-parameter 
* |V|: vocabulary size


* The outcomes of the above calculations shown that in both bi-gram and tri-gram models the probability of the correct sentence is by far higher than the one that derived from the incorrect sentence(random words). Some outcomes of the above calculations are shown below:

In [8]:
# We should fine-tune alpha on a held-out dataset
alpha = 0.01
# Calculate vocab size 
vocab_size = len(set(vocabulary))

''' functions to calculate the log probabilities of  a given correct sentence and an
    incorrect one with Laplace smoothing, based on bigram and trigram models  
'''

def bigram_log_prob(sentence):
    
    seed(10)
    bigram_log_prob = 0
    
    for token in range(len(sentence)-1):
        bigram_log_prob += math.log2((bigram_counter[(sentence[token], sentence[token+1])] + alpha) / (unigram_counter[(sentence[token],)] + alpha*vocab_size))
        
        
    bigram_log_prob_shuffled = 0
    sentence2 = sample(vocabulary, len(sentence))
    
    for token in range(len(sentence2)-1):
        bigram_log_prob_shuffled += math.log2((bigram_counter[(sentence2[token], sentence2[token+1])] +alpha) / (unigram_counter[(sentence2[token],)] + alpha*vocab_size))
        
    
    return "bigram_log_prob: {0:.3f}".format(bigram_log_prob), "bigram_log_prob_shuffled: {0:.3f}".format(bigram_log_prob_shuffled) 

def trigram_log_prob(sentence):
    
    seed(10)
    trigram_log_prob = 0
    
    for token in range(len(sentence)-2):
        trigram_log_prob += math.log2((trigram_counter[(sentence[token], sentence[token+1],sentence[token+2])] +alpha) / (bigram_counter[(sentence[token],sentence[token+1])] + alpha*vocab_size))
        
        
    trigram_log_prob_shuffled = 0
    sentence2 = sample(vocabulary, len(sentence))
    
    for token in range(len(sentence2)-2):
        trigram_log_prob_shuffled += math.log2((trigram_counter[(sentence2[token], sentence2[token+1],sentence2[token+2])] +alpha) / (bigram_counter[(sentence[token],sentence[token+1])] + alpha*vocab_size))
        
    
    return "trigram_log_prob: {0:.3f}".format(trigram_log_prob), "trigram_log_prob_shuffled: {0:.3f}".format(trigram_log_prob_shuffled) 

In [9]:
print("         Correct        VS      Incorrect Sentence")
for i in range(10):
    print("Sentence:", i+1)
    print(bigram_log_prob(test_sentences_tokenized[i]))
    print(trigram_log_prob(test_sentences_tokenized[i]))
    print('-----------------------------------------------------------------')
    
    

         Correct        VS      Incorrect Sentence
Sentence: 1
('bigram_log_prob: -61.582', 'bigram_log_prob_shuffled: -155.573')
('trigram_log_prob: -44.123', 'trigram_log_prob_shuffled: -163.036')
-----------------------------------------------------------------
Sentence: 2
('bigram_log_prob: -408.645', 'bigram_log_prob_shuffled: -906.815')
('trigram_log_prob: -460.414', 'trigram_log_prob_shuffled: -1004.264')
-----------------------------------------------------------------
Sentence: 3
('bigram_log_prob: -240.568', 'bigram_log_prob_shuffled: -734.830')
('trigram_log_prob: -241.286', 'trigram_log_prob_shuffled: -883.140')
-----------------------------------------------------------------
Sentence: 4
('bigram_log_prob: -125.981', 'bigram_log_prob_shuffled: -295.045')
('trigram_log_prob: -157.924', 'trigram_log_prob_shuffled: -326.890')
-----------------------------------------------------------------
Sentence: 5
('bigram_log_prob: -40.432', 'bigram_log_prob_shuffled: -91.820')
('trigra

### iii) Estimate the language cross-entropy and perplexity of your models on the test set
Estimation of language's cross-entropy and perplexity of both models on the test subset of the corpus, treating the entire test subset as a single sequence, with *start* (or *start1*, *start2*) at the beginning of each sentence, and *end* at the end of each sentence.

* Cross Entropy = $ -\frac{\sum{log_2P_{n-gram}}}{N_{n-gram}}$
* Perplexity = $ 2^{Cross Entropy}$
* First we fine tune `alpha` hyperparameter in development set both for bigram and trigram model. 

* `Results of fine-tuning in development set:`

* `Bigram model:`
    - Alpha = 0.003
    - Cross Entropy: 6.387
    - Perplexity   : 83.679
* `Trigram model:`
    - Alpha = 0.002
    - Cross Entropy: 6.642
    - Perplexity   : 99.873
    
* From the results we can see that the Bigram model performs way better in the Development set than the Trigram, meaning that for this corpus the Trigram model overfits. 

In [10]:
# fine tune alpha for bi-gram

alpha = np.arange(0.001,0.01,.001)
print("Fine tune alpha for Bi-gram Model")
print("--------------")
for alpha in alpha:
    sum_prob = 0
    bigram_cnt = 0
    for sent in dev_sentences_tokenized:
        sent = sent + ['*end*']
        for i in range(len(sent)-1):
            bigram_prob = (bigram_counter[(sent[i], sent[i+1])] +alpha) / (unigram_counter[(sent[i],)] + alpha*vocab_size)
            sum_prob += math.log2(bigram_prob)
            bigram_cnt+=1

    HC = -sum_prob / bigram_cnt
    perpl = math.pow(2,HC)
   
    print("Alpha:{0:.3f}".format(alpha))
    print("Cross Entropy: {0:.3f}".format(HC))
    print("perplexity: {0:.3f}".format(perpl))
    print("--------------")

Fine tune alpha for Bi-gram Model
--------------
Alpha:0.001
Cross Entropy: 6.399
perplexity: 84.408
--------------
Alpha:0.002
Cross Entropy: 6.388
perplexity: 83.766
--------------
Alpha:0.003
Cross Entropy: 6.387
perplexity: 83.679
--------------
Alpha:0.004
Cross Entropy: 6.389
perplexity: 83.781
--------------
Alpha:0.005
Cross Entropy: 6.392
perplexity: 83.969
--------------
Alpha:0.006
Cross Entropy: 6.396
perplexity: 84.201
--------------
Alpha:0.007
Cross Entropy: 6.400
perplexity: 84.458
--------------
Alpha:0.008
Cross Entropy: 6.405
perplexity: 84.730
--------------
Alpha:0.009
Cross Entropy: 6.410
perplexity: 85.010
--------------


In [11]:
# fine tune alpha for tri-gram
alpha = np.arange(0.001,0.011,.001)
print("Fine tune alpha for Tri-gram Model")
print("--------------")
for alpha in alpha:
    sum_prob = 0
    trigram_cnt = 0
    for sent in dev_sentences_tokenized:
        sent =  sent +  ['*end*'] + ['*end*']
        for i in range(len(sent)-2):
            trigram_prob = (trigram_counter[(sent[i], sent[i+1],sent[i+2])] +alpha) / (bigram_counter[(sent[i],sent[i+1])] + alpha*vocab_size)
            sum_prob += math.log2(trigram_prob)
            trigram_cnt+=1

    HC = -sum_prob / trigram_cnt
    perpl = math.pow(2,HC)
    print("Alpha:{0:.3f}".format(alpha))
    print("Cross Entropy: {0:.3f}".format(HC))
    print("perplexity: {0:.3f}".format(perpl))
    print("--------------")

Fine tune alpha for Tri-gram Model
--------------
Alpha:0.001
Cross Entropy: 6.607
perplexity: 97.459
--------------
Alpha:0.002
Cross Entropy: 6.642
perplexity: 99.873
--------------
Alpha:0.003
Cross Entropy: 6.687
perplexity: 103.036
--------------
Alpha:0.004
Cross Entropy: 6.731
perplexity: 106.212
--------------
Alpha:0.005
Cross Entropy: 6.772
perplexity: 109.283
--------------
Alpha:0.006
Cross Entropy: 6.810
perplexity: 112.229
--------------
Alpha:0.007
Cross Entropy: 6.846
perplexity: 115.056
--------------
Alpha:0.008
Cross Entropy: 6.880
perplexity: 117.774
--------------
Alpha:0.009
Cross Entropy: 6.912
perplexity: 120.393
--------------
Alpha:0.010
Cross Entropy: 6.942
perplexity: 122.924
--------------


* Thus, we use optimum alpha for each n-model as Laplace smoothing hyper-parameter in order to evaluate the two models on the unknown Test set.

In [12]:
# bi-gram model on test set
alpha_b = 0.003
sum_prob = 0
bigram_cnt = 0
for sent in test_sentences_tokenized:
    sent = sent + ['*end*']
    for i in range(len(sent)-1):
        bigram_prob = (bigram_counter[(sent[i], sent[i+1])] + alpha_b) / (unigram_counter[(sent[i],)] + alpha_b*vocab_size)
        sum_prob += math.log2(bigram_prob)
        bigram_cnt+=1

HC = -sum_prob / bigram_cnt
perpl = math.pow(2,HC)
print("Bi-gram performance on Test set:")
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(perpl))

Bi-gram performance on Test set:
Cross Entropy: 6.401
perplexity: 84.524


In [13]:
# tri-gram model on test set
sum_prob = 0
alpha_t = 0.002
trigram_cnt = 0
for sent in test_sentences_tokenized:
    sent =  sent + ['*end*'] + ['*end*']
    for i in range(len(sent)-2):
        trigram_prob = (trigram_counter[(sent[i], sent[i+1],sent[i+2])] + alpha_t) / (bigram_counter[(sent[i],sent[i+1])] + alpha_t*vocab_size)
        sum_prob += math.log2(trigram_prob)
        trigram_cnt+=1

HC = -sum_prob / trigram_cnt
perpl = math.pow(2,HC)
print("Tri-gram performance on Test set:")
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(perpl))

Tri-gram performance on Test set:
Cross Entropy: 6.666
perplexity: 101.560


### iv) Interpolated bi-gram and tri-gram LM

* We will combine the two models using linear interpolation and check if the combined model performs better.
### $ P(w_3|w_1,w_2) = \lambda \cdot P(w_3|w_1,w_2) +(1-\lambda) \cdot P(w_3|w_2)  $


* $ 0 \leq \lambda \leq 1 $
* We will calculate the cross entropy for several values of $ \lambda $ in order to find out if the interpolated model performs better than the bi-gram. For $ \lambda $=0 the interpolated model takes into account only the bi-gram, while for  $\lambda $=1  only the tri-gram is taken into account.

In [14]:
# fine-tune lamda on development set
lamda = np.arange(0,1.1,0.1)
for lamda in lamda:
    sum_prob = 0
    ngram_cnt = 0
    for sent in dev_sentences_tokenized:
        sent =  sent + ['*end*'] + ['*end*']
        for i in range(len(sent)-2):
            trigram_prob = (trigram_counter[(sent[i], sent[i+1],sent[i+2])] + alpha_t) / (bigram_counter[(sent[i],sent[i+1])] + alpha_t*vocab_size)
            bigram_prob = (bigram_counter[(sent[i], sent[i+1])] + alpha_b) / (unigram_counter[(sent[i],)] + alpha_b*vocab_size)
            
            sum_prob += (lamda * math.log2(trigram_prob)) + ((1-lamda) * math.log2(bigram_prob))
            ngram_cnt+=1 

    HC = -sum_prob / ngram_cnt
    perpl = math.pow(2,HC)
    print("Lamda:{0:.2f}".format(lamda))
    print("Cross Entropy: {0:.3f}".format(HC))
    print("perplexity: {0:.3f}".format(perpl))
    print("---------------")
    

Lamda:0.00
Cross Entropy: 6.387
perplexity: 83.679
---------------
Lamda:0.10
Cross Entropy: 6.412
perplexity: 85.173
---------------
Lamda:0.20
Cross Entropy: 6.438
perplexity: 86.693
---------------
Lamda:0.30
Cross Entropy: 6.463
perplexity: 88.240
---------------
Lamda:0.40
Cross Entropy: 6.489
perplexity: 89.815
---------------
Lamda:0.50
Cross Entropy: 6.514
perplexity: 91.418
---------------
Lamda:0.60
Cross Entropy: 6.540
perplexity: 93.050
---------------
Lamda:0.70
Cross Entropy: 6.565
perplexity: 94.711
---------------
Lamda:0.80
Cross Entropy: 6.591
perplexity: 96.401
---------------
Lamda:0.90
Cross Entropy: 6.616
perplexity: 98.122
---------------
Lamda:1.00
Cross Entropy: 6.642
perplexity: 99.873
---------------


* From the above results on the development set, it can be seen that as the value of $\lambda$ raises, meaning that as the tri-gram model is taken more and more into account, cross entropy raises too. Therefore, the combined model does not perform better than the bi-gram model.